In [4]:
using Pkg
# Pkg.activate(joinpath(pwd(),".."))
# Pkg.add("Optim")
# Pkg.add("ForwardDiff")
# Pkg.add("PrettyTables")
# Pkg.add("Parameters")
# Pkg.add("Distributions")

In [5]:
using Parameters, Optim, ForwardDiff, LinearAlgebra, Distributions, Random, PrettyTables

In [6]:
Random.seed!(123)

MersenneTwister(123)

In [7]:
const beta  = 1
const pi = [1,1]
const gamma = [1,1]
const rho = 0.95
const n = 500
const R = 10000

10000

Define the sample generating function.

In [8]:
function generate_data_Q1(n)
    #Define the Multivariate Normal Distribution instance
    mvnormal = MvNormal([0;0], [1 rho ; rho 1])
    mvnormal_z = MvNormal([0;0], [1 0;0 1])

    #Matrix Z
    Z = rand(mvnormal_z,n)'
    error = rand(mvnormal,n)'

    epsilon = error[:,1]
    V = error[:,2]
    
    U = exp.(Z*gamma).* epsilon

    X = Z * pi + V

    Y = X * beta + U

    return ( Y = Y, X = X , Z = Z)
end

generate_data_Q1 (generic function with 1 method)

Define the 2SLS estimator

In [38]:
function est2SLS(y, x, z)
    n=size(y,1)
    
    #Coef
    Pz = z*inv(z'*z)*z'
    beta2SLS = inv(x'*Pz*x) * (x'*Pz*y)

    # Asymptotic variance and standard error   
    u = y .- x*beta2SLS;
    sd2SLS = abs(inv(x'*z*inv(z'*z)*z'*x)*x'*z*inv(z'*z)*z'u)/sqrt(n)
      
    

    return (beta2SLS=beta2SLS, sd2SLS = sd2SLS)
end

est2SLS (generic function with 1 method)

Define GMM estimator

In [28]:
function estGMM(y,x,z)
    n=size(y,1)

    #First Step
    An = [1 0;0 1]
    betaAn = (x'*z*An*z'*x)^(-1)*x'*z*An*z'*y

    #Second Step
    res = y - x*betaAn
    Omega = res'*z*z'*res.*(1/n)
    betaGMM = (x'*z*inv(Omega)*z'*x)^(-1)*x'*z*inv(Omega)*z'*y

    Q = z'x/n;
    avar = inv((Q'inv(Omega))*Q)
    sdGMM = sqrt(avar)/sqrt(n)

    return (betaGMM =betaGMM, sdGMM = sdGMM)
end

estGMM (generic function with 1 method)

In [24]:
function data(r)
    result = zeros(r,10)
    for i in 1:r
        (Y,X,Z) = generate_data_Q1(n)
        (beta2SLS, sd2SLS) = est2SLS(Y,X,Z)
        (betaGMM, sdGMM) = estGMM(Y,X,Z)
        result[i,1] = abs(beta2SLS - beta)
        result[i,2] = abs(betaGMM - beta)
        result[i,3] = beta2SLS - sd2SLS
        result[i,4] = beta2SLS + sd2SLS
        result[i,5] = betaGMM -  sdGMM
        result[i,6] = betaGMM + sdGMM
        result[i,7] = sd2SLS
        result[i,8] = sdGMM
        if result[i,1] < 1.96*result[i,7] 
            result[i,9] = 1
        else
            result[i,9] = 0
        end
    
        if result[i,2] < 1.96*result[i,8]
            result[1,10] = 1
        else
            result[i,10] = 0
        end
    end

    return (result = result)
end

data (generic function with 1 method)

In [39]:
result = data(R)
table= ["Average Bias" round(mean(result[:,1]), digits = 7)  round(mean(result[:,2]), digits = 7); "Average S.D." round(mean(result[:,7]), digits = 7) round(mean(result[:,8]), digits = 7); "Coverage Probability" round(mean(result[:,9]), digits = 7)    round(mean(result[:,10]), digits = 7)]

header = [" ", "2SLS", "GMM"]
pretty_table(table; header = header)

┌──────────────────────┬─────────┬──────────┐
│                      │    2SLS │      GMM │
├──────────────────────┼─────────┼──────────┤
│         Average Bias │ 0.45754 │ 0.457627 │
│         Average S.D. │     0.0 │ 0.169037 │
│ Coverage Probability │     0.0 │   0.0001 │
└──────────────────────┴─────────┴──────────┘
